In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib.request 
import pandas as pd

#Iin this code I am srapping the relavent links to files from certain dates - here I am extracting one link as an example. 

URL = "https://www.theyworkforyou.com/pwdata/scrapedxml/debates/"

response = urllib.request.urlopen(URL)
soup = BeautifulSoup(response.read())

# Retrieve the url links to each debate file.
all_urls = []
for link in soup.find_all('a', href=lambda href: href and 'debates2019-12-19c' in href):
    if '.xml' in link['href']:
        all_urls.append(URL + link['href'])
        
for url in all_urls:
    print(url)
    
from bs4 import BeautifulSoup, SoupStrainer
import urllib.request 
import pandas as pd

# Reading the files and extracting the rows/fields of interest.
final_dataset = []
for u in all_urls:
    filename = u.split('/')[-1]
    urllib.request.urlretrieve(u, filename)
    with open(filename, 'r', encoding='utf-8') as f:
        xml = f.read()
        soup = BeautifulSoup(xml , 'lxml')
        for speech in soup.find_all('speech'):
            if speech.get('speakername') is not None:     
                name = speech.get('speakername')
                speechid = speech.get('id')
                pid = speech.get('person_id')
                text = speech.find('p').get_text()
                final_dataset.append((speechid, name, pid, text))

# Saving the dataset in a csv format.
df = pd.DataFrame(final_dataset, columns=['speech_id','name', 'person_id', 'text'])
df.to_csv('extra_row.csv', sep=',', index=False)

https://www.theyworkforyou.com/pwdata/scrapedxml/debates/debates2019-12-19c.xml


In [2]:
import pandas as pd

#this merges the extracted files from 2019-2021 and 2022

# read the two CSV files
df0 = pd.read_csv('extra_row.csv')
df1 = pd.read_csv('uk_hansard_2020.csv')
df2 = pd.read_csv('uk_hansard_2021.csv')

# merge the two dataframes
merged_df = pd.concat([df0, df1, df2])

# write the merged dataframe to a new CSV file
merged_df.to_csv('hansard2020_2021.csv', index=False)

df_mp = pd.read_csv('mps.csv')
#df_mp['name'] = df_mp['First name'].str.cat(df_mp['Last name'], sep=' ')

# print the updated dataframe
df_mp.head()

df_mp.tail()

,Person ID,First name,Last name,Party,Constituency,URI
645,11791,Jeremy,Wright,Conservative,Kenilworth and Southam,https://www.theyworkforyou.com/mp/11791/jeremy...
646,25649,Mohammad,Yasin,Labour,Bedford,https://www.theyworkforyou.com/mp/25649/mohamm...
647,25806,Jacob,Young,Conservative,Redcar,https://www.theyworkforyou.com/mp/25806/jacob_...
648,24822,Nadhim,Zahawi,Conservative,Stratford-on-Avon,https://www.theyworkforyou.com/mp/24822/nadhim...
649,25386,Daniel,Zeichner,Labour,Cambridge,https://www.theyworkforyou.com/mp/25386/daniel...


In [6]:
# making a standardised person id column in this section.
merged_df['Person ID'] = merged_df['person_id'].str.rsplit('/', n=1).str[-1]

merged_df.head()

merged_df['Person ID'] = merged_df['Person ID'].fillna(-1).astype('int64')
merged_data = pd.merge(merged_df, df_mp, on='Person ID',  how= 'left')
print(merged_df.dtypes)
#print(df_mp.dtypes)
# add the extracted info to a new excel doc.
merged_data.to_csv('updated_speech_data3.csv', index=False)

#here I am doing the same thing but for 2022-2023 debates - upto the last debate on the link at the time which was april 2023
df23 = pd.read_csv('uk_hansard_2022_2023.csv')
df23['Person ID'] = df23['person_id'].str.rsplit('/', n=1).str[-1]
df23.head()
df23['Person ID'] = df23['Person ID'].fillna(-1).astype('int64')
merged_data2 = pd.merge(df23, df_mp, on='Person ID',  how= 'left')
print(df23.dtypes)
print(df_mp.dtypes)
#merged_data2.head()

speech_id    object
name         object
person_id    object
text         object
Person ID     int64
dtype: object
speech_id    object
name         object
person_id    object
text         object
Person ID     int64
dtype: object
Person ID        int64
First name      object
Last name       object
Party           object
Constituency    object
URI             object
dtype: object


In [7]:
#here I am creating a dataframe whic combines the 2020 to 2023 data. 
merged_2020_2023 = pd.concat([merged_data, merged_data2])
#merged_2020_2023.to_csv('debates_2020_23.csv', index=False)
merged_2020_2023.tail()

,speech_id,name,person_id,text,Person ID,First name,Last name,Party,Constituency,URI
81584,uk.org.publicwhip/debate/2023-04-26a.896.3,Robert Jenrick,uk.org.publicwhip/person/25227,"Perhaps I can answer some of those questions, ...",25227,Robert,Jenrick,Conservative,Newark,https://www.theyworkforyou.com/mp/25227/robert...
81585,uk.org.publicwhip/debate/2023-04-26a.897.0,Richard Drax,uk.org.publicwhip/person/24903,Can the Minister confirm that this Stockholm B...,24903,Richard,Drax,Conservative,South Dorset,https://www.theyworkforyou.com/mp/24903/richar...
81586,uk.org.publicwhip/debate/2023-04-26a.898.0,Robert Jenrick,uk.org.publicwhip/person/25227,I am not sure of the source to which my hon. F...,25227,Robert,Jenrick,Conservative,Newark,https://www.theyworkforyou.com/mp/25227/robert...
81587,uk.org.publicwhip/debate/2023-04-26a.898.1,Chris Loder,uk.org.publicwhip/person/25918,"As my right hon. Friend will know, my neighbou...",25918,Chris,Loder,Conservative,West Dorset,https://www.theyworkforyou.com/mp/25918/chris_...
81588,uk.org.publicwhip/debate/2023-04-26a.898.2,Robert Jenrick,uk.org.publicwhip/person/25227,"I know that my hon. Friend met my officials, a...",25227,Robert,Jenrick,Conservative,Newark,https://www.theyworkforyou.com/mp/25227/robert...


In [8]:
#here i am cleaning up and standardising the format for each of the columns and creating an standardised date column.
import re
merged_2020_2023['speech_id_str'] = merged_2020_2023['speech_id'].astype(str)
merged_2020_2023['speech_id_str'] = merged_2020_2023['speech_id_str'].fillna('')
merged_2020_2023['Date'] = merged_2020_2023['speech_id_str'].apply(lambda x: x.split('/')[2])
merged_2020_2023['Date'] = merged_2020_2023['Date'].str[:10]

merged_2020_2023['Date'] = pd.to_datetime(merged_2020_2023['Date'])
merged_2020_2023['Date'] = merged_2020_2023['Date'].dt.strftime('%d/%m/%Y')
merged_2020_2023.head()

,speech_id,name,person_id,text,Person ID,First name,Last name,Party,Constituency,URI,speech_id_str,Date
0,uk.org.publicwhip/debate/2019-12-19c.27.3,Lindsay Hoyle,uk.org.publicwhip/person/10295,Will Members wishing to take their seats pleas...,10295,Lindsay,Hoyle,Speaker,Chorley,https://www.theyworkforyou.com/mp/10295/lindsa...,uk.org.publicwhip/debate/2019-12-19c.27.3,19/12/2019
1,uk.org.publicwhip/debate/2019-12-19c.27.4,Lindsay Hoyle,uk.org.publicwhip/person/10295,The sitting is suspended until 2.30 pm. The be...,10295,Lindsay,Hoyle,Speaker,Chorley,https://www.theyworkforyou.com/mp/10295/lindsa...,uk.org.publicwhip/debate/2019-12-19c.27.4,19/12/2019
2,uk.org.publicwhip/debate/2019-12-19c.27.6,Lindsay Hoyle,uk.org.publicwhip/person/10295,"Colleagues, I want to begin this Session by ma...",10295,Lindsay,Hoyle,Speaker,Chorley,https://www.theyworkforyou.com/mp/10295/lindsa...,uk.org.publicwhip/debate/2019-12-19c.27.6,19/12/2019
3,uk.org.publicwhip/debate/2019-12-19c.28.0,Barry Sheerman,uk.org.publicwhip/person/10534,"On a point of order, Mr Speaker.",10534,Barry,Sheerman,Labour/Co-operative,Huddersfield,https://www.theyworkforyou.com/mp/10534/barry_...,uk.org.publicwhip/debate/2019-12-19c.28.0,19/12/2019
4,uk.org.publicwhip/debate/2019-12-19c.28.1,Lindsay Hoyle,uk.org.publicwhip/person/10295,Not at this stage.,10295,Lindsay,Hoyle,Speaker,Chorley,https://www.theyworkforyou.com/mp/10295/lindsa...,uk.org.publicwhip/debate/2019-12-19c.28.1,19/12/2019


The file below was taken from: https://reshare.ukdataservice.ac.uk/854292/ it is data taken from Jack Blumenau's (2021) dataset which covers all parliamentary speeches I took the speeches from 1988 to near the end of 2019.

In [9]:
speeches = pd.read_csv('speech_data.csv')
speeches.tail()

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
1956218,2019-12-17,Election of Speaker,14,Iain Stewart,Con,1567.0,False,34,I have to signify to the House the pleasure of...,UK-HouseOfCommons,GBR
1956219,2019-12-17,Election of Speaker,15,CHAIR,NaN,NaN,True,7,The House is suspended until 3.45 pm.,UK-HouseOfCommons,GBR
1956220,2019-12-17,MESSAGE TO ATTEND THE LORDS COMMISSIONERS,16,CHAIR,NaN,NaN,True,82,I have to report that this House has been in t...,UK-HouseOfCommons,GBR
1956221,2019-12-17,Members Sworn,17,CHAIR,NaN,NaN,True,54,"I must now call on the Members of the House, a...",UK-HouseOfCommons,GBR
1956222,2019-12-17,Members Sworn,18,CHAIR,NaN,NaN,True,1709,"I should remind hon. Members that, until they ...",UK-HouseOfCommons,GBR


In [10]:
## this section of the code merges the two datasets together across the desired columns
combined = pd.concat([speeches['speaker'], merged_2020_2023['name']], ignore_index=True)
df_combined = pd.DataFrame({'speaker2': combined})

combined2 = pd.concat([speeches['text'], merged_2020_2023['text']], ignore_index=True)
df_combined2 = pd.DataFrame({'text2': combined2})

combined3 = pd.concat([speeches['party'], merged_2020_2023['Party']], ignore_index=True)
df_combined3 = pd.DataFrame({'party2': combined3})

combined4 = pd.concat([speeches['date'], merged_2020_2023['Date']], ignore_index=True)
df_combined4 = pd.DataFrame({'date2': combined4})

df_combined_all = pd.concat([df_combined, df_combined2, df_combined3, df_combined4], axis=1)
df_combined_all.tail()

data_1988_2023 = pd.concat([speeches, df_combined_all], axis=1)
data_1988_2023.head()

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,speaker2,text2,party2,date2
0,1988-11-22,Queen's Speech,1.0,CHAIR,NaN,NaN,True,74.0,I have to acquaint the House that this House h...,UK-HouseOfCommons,GBR,CHAIR,I have to acquaint the House that this House h...,NaN,1988-11-22
1,1988-11-22,First Day [Debate On The Address],2.0,CHAIR,NaN,NaN,True,55.0,It may be for the convenience of the House if ...,UK-HouseOfCommons,GBR,CHAIR,It may be for the convenience of the House if ...,NaN,1988-11-22
2,1988-11-22,First Day [Debate On The Address],3.0,Giles Shaw,Con,1567.0,False,2511.0,"I beg to move,. That an humble Address be pres...",UK-HouseOfCommons,GBR,Giles Shaw,"I beg to move,. That an humble Address be pres...",Con,1988-11-22
3,1988-11-22,First Day [Debate On The Address],4.0,John Maples,Con,1567.0,False,1470.0,I am delighted to second the motion. When I ha...,UK-HouseOfCommons,GBR,John Maples,I am delighted to second the motion. When I ha...,Con,1988-11-22
4,1988-11-22,First Day [Debate On The Address],5.0,Neil Kinnock,Lab,1516.0,False,2768.0,I am sure that I speak for the majority of hon...,UK-HouseOfCommons,GBR,Neil Kinnock,I am sure that I speak for the majority of hon...,Lab,1988-11-22


In [11]:
#this code checks the data type, the size of the data and drops repeated or unimportant columns 
data_3 = data_1988_2023.drop(columns=['date', 'speaker', 'party', 'text', 'parliament', 'iso3country'])
data_3.head()
data_3['text2'] = data_3['text2'].astype(str)
print(data_3.dtypes)
data_3.head()
num_rows = len(data_3)
print('Number of rows:', num_rows)

agenda             object
speechnumber      float64
party.facts.id    float64
chair              object
terms             float64
speaker2           object
text2              object
party2             object
date2              object
dtype: object
Number of rows: 2152323


In [12]:
#creating standardised names for the core political parties across the whole dataset. 
df_filtered3= data_3.copy()
df = df_filtered3.copy()
df['party2'] = df['party2'].astype(str)
df['party2'] = df['party2'].str.replace('Labour', 'Lab')
df['party2'] = df['party2'].str.replace('Conservative', 'Con')
df['party2'] = df['party2'].str.replace('Scottish National Party', 'SNP')
df['party2'] = df['party2'].str.replace('Liberal Democrat', 'LibDem') 

# Standardisw the date column to "DD/MM/YYYY" format
df['date2'] = pd.to_datetime(df['date2'], dayfirst=True).dt.strftime('%d/%m/%Y')
df['Year'] = pd.to_datetime(df['date2'], format='%d/%m/%Y').dt.year

# Create 'speech_id' this will hold the index values of the speeches 
#from the index column to help see where text came from
#once the texts are split up into references. 
df['speech_id'] = df.index

#Drop uniformative columns for the dataset
columns_to_drop = ['party.facts.id', 'chair']
df.drop(columns_to_drop, axis=1, inplace=True)
df.head(50)

,agenda,speechnumber,terms,speaker2,text2,party2,date2,Year,speech_id
0,Queen's Speech,1.0,74.0,CHAIR,I have to acquaint the House that this House h...,nan,22/11/1988,1988,0
1,First Day [Debate On The Address],2.0,55.0,CHAIR,It may be for the convenience of the House if ...,nan,22/11/1988,1988,1
2,First Day [Debate On The Address],3.0,2511.0,Giles Shaw,"I beg to move,. That an humble Address be pres...",Con,22/11/1988,1988,2
3,First Day [Debate On The Address],4.0,1470.0,John Maples,I am delighted to second the motion. When I ha...,Con,22/11/1988,1988,3
4,First Day [Debate On The Address],5.0,2768.0,Neil Kinnock,I am sure that I speak for the majority of hon...,Lab,22/11/1988,1988,4
5,First Day [Debate On The Address],6.0,1.0,David Harris,Appeal.,Con,22/11/1988,1988,5
6,First Day [Debate On The Address],7.0,850.0,Neil Kinnock,"The hon. Gentleman says, "" Appeal."" Of course ...",Lab,22/11/1988,1988,6
7,First Day [Debate On The Address],8.0,189.0,Margaret Thatcher,"First, I join the Leader of the Opposition in ...",Con,22/11/1988,1988,7
8,First Day [Debate On The Address],9.0,23.0,CHAIR,Order. I did not wish to interrupt the Leader ...,nan,22/11/1988,1988,8
9,First Day [Debate On The Address],10.0,37.0,Margaret Thatcher,Let me deal quickly with some of the comments ...,Con,22/11/1988,1988,9


FILTERING THE DATASET SO THAT IT ONLY CONTAINS TEXTS RELAVENT TO CHINA AND CREATING THE REFERENCES TO CHINA CORPUS.

In [13]:
import string 
#filtering the dataset based on the two keywords - china or chinese.
df['text'] = df['text2'].str.lower()
df2 = df.copy()
df2 = df2[df2['text'].str.contains('china|chinese', case=False)]
print(len(df2))

15087


In [14]:
#this code removes short texts that are less than 30 words longs and also removes from texts certain words which are overly
#common to parliamentary speeches. 
#this code is addapted from that shown in the youtube video here:  

df3 = df2.copy()
# remove specific words and phrases from text
def remove_words(text, words_to_remove):
    for word in words_to_remove:
        text = text.replace(word, '')
    return text.strip()

# Define the words and phrases to remove
words_to_remove = ['right hon. member', 'hon.', 'hon. member', 'right hon. friend', 'hon. friend', 'honorable member',
                   'honorable friend', 'mr. speaker', 'madam speaker', 'deputy speaker', 'right honorable member',
                   'right honorable friend', 'the member for']

# Apply the remove_words function to the text column
df2['text'] = df2['text'].apply(lambda x: remove_words(x, words_to_remove))

# Filter out rows with less than 50 words
df3 = df2[df2['text'].apply(lambda x: len(x.split()) >= 30)]

print(len(df3))

14765


this section of the code adapts the blocks of three sentences used in this:
1. https://stackoverflow.com/questions/59178739/tokenizing-text-by-3-sentences-using-nltk-and-pandas
the code makes use of techniques in this resource:
2. https://www.nltk.org/book/ and https://www.nltk.org/book/ch01.html#sec-text-mining

In [24]:
import nltk
from nltk.tokenize import sent_tokenize
import re


df6 = df3.copy()

# searching for the keywords. 
keyword_china = 'china'
keyword_chinese = 'chinese'

# I am creating an empty list to store each of the references
references = []
references_found = set()

# start by interating over  each of the rows of the DataFrame
for _, row in df6.iterrows():
    # this checks that all the text has been made lower case.
    text = row['text']
    preprocessed_text = text.lower()

    # Tokenize the text into sentences using NLTK
    sentences = sent_tokenize(preprocessed_text)

    # Check if the speech has enough sentences for a reference
    if len(sentences) >= 3:
        # This second for for loop sees if the sentence containing the key word is the last or second last in a block.
        for i, sentence in enumerate(sentences[:-2]):
            #this makes sure that only sentences with the word china or chinese are taken.
            if keyword_china in sentence or keyword_chinese in sentence:
                # This third if statement checks if the sentence has already been included in a reference
                if sentence not in references_found:
                    # this says if no keywords found check the next sentence in the text.
                    reference_sentences = [sentence]
                    current_sentence_index = i + 1

                    # Here I am adding the next two sentences to the reference
                    while len(reference_sentences) < 3 and current_sentence_index < len(sentences):
                        next_sentence = sentences[current_sentence_index]

                        # Here I am using an if statement to check whether the next sentence is already included in a reference
                        if next_sentence not in references_found:
                            reference_sentences.append(next_sentence)
                            references_found.add(next_sentence)

                        current_sentence_index += 1

                    # Here the 3 sentences are joined together to make a single reference which is a single string
                    reference = " ".join(reference_sentences)

                    # Here the information for each reference is appended into the empty references list. 
                    reference_info = {
                        'Reference to China': reference,
                        'speaker2': row['speaker2'],
                        'speech_id': row['speech_id'],
                        'speechnumber': row['speechnumber'],
                        'party2': row['party2'],
                        'agenda': row['agenda'],
                        'date2': row['date2'],
                        'Year': row['Year']
                    }
                    references.append(reference_info)

# Create a new dataframe with the references
df_references = pd.DataFrame(references)
print(len(df_references))
df_references.head(15) 

df_references['Word Count'] = df_references['Reference to China'].apply(lambda x: len(x.split()))
shortest = df_references['Word Count'].quantile(0.015)
print(shortest)

##this code does not yet produce the final dataset as here references to China that are longer the 30
#tokens are still included, these are removed in the following document. Hence, why that are 138 more references to China 
#than in the final dataset. # it shows the removing texts shorter than 30 removes outliers - the shortest 1.5% of texts.

15338
32.0


In [16]:
# Filter out rows with less than 50 words
df_references = df_references[df_references['Reference to China'].apply(lambda x: len(x.split()) >= 30)]

print(len(df_references))

15200


In [ ]:
#this is the final csv final for the references to China dataset - this is a novel contribution to the field of text as data.

#df_references.to_csv('references.csv', index=False) 

#uncomment the above to save it. 